In [1]:
# =============================
# Earth Engine Python - Paddy ML dataset (Sri Lanka)
# =============================

# 0) Setup
try:
    import ee
except ModuleNotFoundError:
    !pip -q install earthengine-api geemap
    import ee

try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

# -----------------------------
# 1) User inputs (EDIT THESE)
# -----------------------------
START = '2024-01-01'
END   = '2024-12-31'
CLOUD_PROB_THRESHOLD = 40     # 0-100
S1_DAY_WINDOW = 3             # +/- days around each S2 date
EXPORT_DESCRIPTION = 'paddy_ml_dataset_full'
EXPORT_FOLDER = 'earthengine' # Google Drive folder

# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Load AOI (field area) from a GeoJSON file
# Replace the file path with your downloaded GeoJSON file path or Drive mount path.
# Example: '/content/drive/MyDrive/paddy_AOI_geojson.geojson'
import json

geojson_path = '/content/sample_data/AOI_Geometry.geojson'  # <-- REPLACE THIS

with open(geojson_path, 'r') as f:
    geojson_data = json.load(f)

geometry = ee.Geometry(geojson_data['features'][0]['geometry'])
print('Loaded AOI from GeoJSON ✅')
print('Area (ha):', geometry.area().divide(10000).getInfo())
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<


# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# geometry: PUT YOUR FIELD/AREA HERE (GeoJSON or coords). Placeholder below.
# Example bbox for testing; REPLACE with your AOI polygon.
# geometry = ee.Geometry.Polygon([[[80.0,7.2],[80.0,7.1],[80.1,7.1],[80.1,7.2],[80.0,7.2]]])
# geometry = ee.Geometry.BBox(79.95, 7.00, 80.00, 7.05)  # <-- REPLACE THIS
# <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

# -----------------------------
# 2) Datasets (FREE)
# -----------------------------
# Sentinel-2 SR + Cloud Prob
S2   = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(geometry).filterDate(START, END))
S2CP = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(geometry).filterDate(START, END))

# Sentinel-1 (VV/VH, IW)
S1 = (ee.ImageCollection('COPERNICUS/S1_GRD')
      .filterBounds(geometry)
      .filterDate(ee.Date(START).advance(-S1_DAY_WINDOW,'day'),
                  ee.Date(END).advance(S1_DAY_WINDOW,'day'))
      .filter(ee.Filter.eq('instrumentMode', 'IW'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
      .map(lambda img: (img
        .addBands(img.select('VV').log10().multiply(10).rename('S1_VV_dB'))
        .addBands(img.select('VH').log10().multiply(10).rename('S1_VH_dB'))
        .copyProperties(img, img.propertyNames()))))

# CHIRPS (mm/day)
CHIRPS = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
          .filterBounds(geometry).filterDate(START, END))

# ERA5-Land daily aggregated (K, J/m2, m, m/s)
ERA5 = (ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
        .filterBounds(geometry).filterDate(START, END))

# SRTM elevation → slope & aspect
SRTM = ee.Image('USGS/SRTMGL1_003').rename('elev')
terrain = ee.Terrain.products(SRTM)
SLOPE = terrain.select('slope').rename('slope_deg')
ASPECT = terrain.select('aspect').rename('aspect_deg')

# ESA WorldCover (10 m) class; 40 = cropland
WORLDCOVER = ee.Image('ESA/WorldCover/v200/2021').select('Map').rename('worldcover')

# SoilGrids v2.0 (topsoil ~0–5 cm)
# Available band names commonly used:
# clay_0-5cm_mean, sand_0-5cm_mean, soc_0-5cm_mean, phh2o_0-5cm_mean
SOIL = (ee.Image('projects/soilgrids-isric/soilgrids/v2.0/soilgrids_mean')
        .select([
            'clay_0-5cm_mean','sand_0-5cm_mean','soc_0-5cm_mean','phh2o_0-5cm_mean'
        ], [
            'soil_clay_pct','soil_sand_pct','soil_soc_gkg','soil_ph_h2o'
        ]))

# -----------------------------
# 3) Sentinel-2: join, mask, indices
# -----------------------------
join = ee.Join.saveFirst('clouds')
onId = ee.Filter.equals(leftField='system:index', rightField='system:index')
S2_joined = join.apply(S2, S2CP, onId)

def mask_s2(img):
    cp  = ee.Image(img.get('clouds')).select('probability')
    scl = ee.Image(img).select('SCL')
    cloud_mask = cp.lt(CLOUD_PROB_THRESHOLD)
    scl_mask = (scl.neq(3)  # cloud shadow
                .And(scl.neq(8))  # cloud
                .And(scl.neq(9))  # cirrus
                .And(scl.neq(10)) # snow
                .And(scl.neq(11)) # no data
               )
    return ee.Image(img).updateMask(cloud_mask).updateMask(scl_mask)

def add_indices(img):
    b2, b3, b4, b8, b11 = [img.select(b) for b in ['B2','B3','B4','B8','B11']]
    ndvi = b8.subtract(b4).divide(b8.add(b4)).rename('NDVI')
    evi  = img.expression('2.5*((NIR-RED)/(NIR+6*RED-7.5*BLUE+1))',
                          {'NIR':b8,'RED':b4,'BLUE':b2}).rename('EVI')
    ndwi = b8.subtract(b11).divide(b8.add(b11)).rename('NDWI')
    savi = b8.subtract(b4).multiply(1.5).divide(b8.add(b4).add(0.5)).rename('SAVI')
    return img.addBands([ndvi, evi, ndwi, savi])

S2_clean = (ee.ImageCollection(S2_joined)
            .map(mask_s2)
            .map(add_indices)
            .select(['NDVI','EVI','NDWI','SAVI','B2','B3','B4','B8','B11','B12','SCL'])
           )

# -----------------------------
# 4) Helpers for ERA5 (derived vars)
# -----------------------------
def era5_enrich_dict(date):
    """Return a dict of ERA5 daily means over AOI for 'date' (UTC day)."""
    d0 = ee.Date(date); d1 = d0.advance(1,'day')
    img = ERA5.filterDate(d0, d1).first()
    empty = ee.Dictionary({})
    stats = ee.Dictionary(ee.Algorithms.If(img, img.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=geometry, scale=5000, maxPixels=1e9
    ), empty))

    # Raw means
    t2m_K   = ee.Number(stats.get('temperature_2m'))
    td_K    = ee.Number(stats.get('dewpoint_temperature_2m'))
    ssrd_Jm2 = ee.Number(stats.get('surface_solar_radiation_downwards_sum'))
    u10 = ee.Number(stats.get('u_component_of_wind_10m'))
    v10 = ee.Number(stats.get('v_component_of_wind_10m'))
    tp_m = ee.Number(stats.get('total_precipitation_sum'))  # optional

    # Conversions / derived
    t2m_C = t2m_K.subtract(273.15)
    td_C  = td_K.subtract(273.15)
    # Relative humidity approximation using Magnus formula
    def magnus(x):  # x in °C
        return (ee.Number(17.625).multiply(x)
               ).divide(ee.Number(243.04).add(x))
    rh = ee.Number(100).multiply( (magnus(td_C).exp()).divide(magnus(t2m_C).exp()) )  # %
    rh = rh.max(0).min(100)

    ssrd_MJm2 = ssrd_Jm2.divide(1e6)         # J/m2 -> MJ/m2
    wind_ms = (u10.pow(2).add(v10.pow(2))).sqrt()
    tp_mm = tp_m.multiply(1000)              # m -> mm (optional)

    return ee.Dictionary({
        'era_temp_C': t2m_C,
        'era_rh_pct': rh,
        'era_ssrd_MJm2': ssrd_MJm2,
        'era_wind_ms': wind_ms,
        'era_tp_mm': tp_mm
    })

# -----------------------------
# 5) Per-date feature builder
# -----------------------------
def feature_for_s2(img):
    date = ee.Date(img.get('system:time_start'))
    d0 = date; d1 = date.advance(1,'day')
    date_str = d0.format('YYYY-MM-dd')
    doy = d0.getRelative('day', 'year').add(1)  # 1..366
    month = d0.get('month')

    # S2 stats (mean & std)
    s2_stats = img.reduceRegion(
        reducer=ee.Reducer.mean().combine(reducer2=ee.Reducer.stdDev(), sharedInputs=True),
        geometry=geometry, scale=10, maxPixels=1e10
    )

    # CHIRPS (same day)
    chirps = CHIRPS.filterDate(d0, d1).first()
    rain_mm = ee.Algorithms.If(chirps, chirps.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=geometry, scale=5000, maxPixels=1e9
    ).get('precipitation'), None)

    # S1 window median
    s1_med = (S1.filterDate(d0.advance(-S1_DAY_WINDOW,'day'),
                            d0.advance(S1_DAY_WINDOW+1,'day')).median())
    s1_stats = s1_med.reduceRegion(
        reducer=ee.Reducer.mean(), geometry=geometry, scale=10, maxPixels=1e10
    )

    # Static: elevation, slope, aspect, soil, worldcover
    elev = SRTM.reduceRegion(ee.Reducer.mean(), geometry, 30, 1e9).get('elev')
    slope = SLOPE.reduceRegion(ee.Reducer.mean(), geometry, 30, 1e9).get('slope_deg')
    aspect = ASPECT.reduceRegion(ee.Reducer.mean(), geometry, 30, 1e9).get('aspect_deg')
    soil_stats = SOIL.reduceRegion(ee.Reducer.mean(), geometry, 250, 1e9)
    worldcover_mode = WORLDCOVER.reduceRegion(ee.Reducer.mode(), geometry, 10, 1e10).get('worldcover')

    # ERA5 daily
    era = era5_enrich_dict(d0)

    # Assemble feature
    feat = ee.Feature(None, s2_stats) \
        .set('date', date_str) \
        .set('doy', doy) \
        .set('month', month) \
        .set('rain_mm', rain_mm) \
        .set('S1_VV_dB', s1_stats.get('S1_VV_dB')) \
        .set('S1_VH_dB', s1_stats.get('S1_VH_dB')) \
        .set('elev_m', elev) \
        .set('slope_deg', slope) \
        .set('aspect_deg', aspect) \
        .set('worldcover', worldcover_mode) \
        .setMulti(soil_stats) \
        .setMulti(era)
    return feat

table = S2_clean.map(feature_for_s2).sort('date')

# -----------------------------
# 6) Add ΔNDVI and ΔEVI (time derivatives)
# -----------------------------
def add_deltas(fc):
    lst = fc.toList(fc.size())
    def step(prev, curr):
        prev = ee.List(prev)
        curr = ee.Feature(curr)
        i = prev.length()
        # get previous NDVI/EVI
        prev_feat = ee.Feature(ee.Algorithms.If(i.eq(0), None, prev.get(-1)))
        curr_ndvi = ee.Number(curr.get('NDVI_mean'))
        curr_evi  = ee.Number(curr.get('EVI_mean'))
        delta_ndvi = ee.Algorithms.If(prev_feat, curr_ndvi.subtract(ee.Number(prev_feat.get('NDVI_mean'))), None)
        delta_evi  = ee.Algorithms.If(prev_feat, curr_evi.subtract(ee.Number(prev_feat.get('EVI_mean'))), None)
        curr = curr.set({'dNDVI': delta_ndvi, 'dEVI': delta_evi})
        return prev.add(curr)
    out_list = ee.List(lst.iterate(step, ee.List([])))
    return ee.FeatureCollection(out_list)

table = add_deltas(table)

# -----------------------------
# 7) Preview & Export
# -----------------------------
print('Preview first 5 rows (properties only):')
try:
    print(table.limit(5).getInfo())
except Exception as e:
    print('Preview skipped:', e)

task = ee.batch.Export.table.toDrive(
    collection=table,
    description=EXPORT_DESCRIPTION,
    folder=EXPORT_FOLDER,
    fileFormat='CSV'
)
task.start()
print(f'Export started: {EXPORT_DESCRIPTION}. Check Earth Engine Tasks / Google Drive.')


EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.